<a href='http://www.holoviews.org'><img src="assets/hv+bk.png" alt="HV+BK logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>07. GeoViews: Working with geographic data</h2></div>

The geoviews package provides a library of Element types which extend standard HoloViews functionality by making the elements aware of geographic projection. The plotting code will automatically transform coordinates to the appropriate projection during plotting so you can work with latitudes and longitudes while GeoViews will handle the complexities of projecting data to mercator coordinates.

In [ ]:
import cartopy.crs as ccrs
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf

hv.extension('bokeh', 'matplotlib')

## Projections

The Cartopy project provides a convenient Python wrapper around the proj4 library making it easy to define geographic projections in Python. GeoViews elements accept a ``crs`` parameter defining the coordinate reference system the data defined on the element is defined as. By default all GeoViews elements assume the ``PlateCarree`` projection which is a Equirectangular projection with coordinates defined as regular latitudes and longitudes. When plotting geographic data with bokeh GeoViews will automatically convert the data to WebMercator coordinates allowing it to be easily overlaid on top of tiles sources.

In [ ]:
%%opts Points [width=600 height=500] (size=6)

nyc = (-74, 40.7, 'NYC')
london = (0.1, 51.5, 'London')
beijing = (116.4, 39.9, 'Beijing')

points = gv.Points([nyc, london, beijing], vdims=['City'], extents=(-180, -90, 180, 90))
print('Default projection:', points.crs)
points * gf.borders * gf.coastline

When plotting with matplotlib we have complete freedom about the projection we want to display the data in, independent of the projection of the original data. Here we switch to the ``Robinson`` projection:

In [ ]:
%%output backend='matplotlib' size=250 fig='svg'
%%opts Points [projection=ccrs.Robinson()]
points * gf.borders * gf.coastline

## Projecting data

GeoViews also provides operations to easily project data (which is what the plotting classes use in the background). Here we project our points to the Google Mercator projection and display them as a pandas DataFrame:

In [ ]:
projected = gv.operation.project(points)
print('New projection:', projected.crs)
projected.dframe().set_index('City')

It is often useful to project the data ahead of time, particularly for large datasets, because when working interactively we often display the data many times and don't want to reproject it every time.

## Tile sources

Tile sources are very convenient ways to provide geographic context for a plot and they will be familiar from the popular mapping services such as Google Maps and Openstreetmap. The ``WMTS`` element provides an easy way to include such a tile source in your visualization simply by passing it a valid URL template. Here we provide a list of common tile sources for use with Bokeh. Additional open tile sources you could use can be found at [openstreetmap.org](http://wiki.openstreetmap.org/wiki/Tile_servers).

In [ ]:
%%opts WMTS [width=400 height=300 xaxis=None yaxis=None]
from bokeh.tile_providers import STAMEN_TONER

tiles = {'OpenMap': 'http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png',
         'ESRI': 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/'
                 'MapServer/tile/{Z}/{X}/{Y}.jpg',
         'Wikipedia': 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}.png',
         'Stamen Toner': STAMEN_TONER}

hv.Layout([gv.WMTS(wmts, extents=(0, -90, 360, 90), label=name)
           for name, wmts in tiles.items()]).cols(2)

## Geometries

#### Features

As we have already discovered GeoViews ships with a number of geographic features available under ``geoviews.feature``, which we have imported as ``gf``. Additionally we may load other ``NaturalEarthFeature``s that can be loaded using ``cartopy``:

In [ ]:
%%opts Feature [width=600 height=500]
import cartopy.feature as cf
graticules = cf.NaturalEarthFeature(
    category='physical',
    name='graticules_30',
    scale='110m')
gf.ocean*gf.land*gf.coastline* gv.Feature(graticules, group='Lines')

#### Geopandas

GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types, which makes it a very convenient way of working with geometries with associated variables. A GeoPandas dataframe acts just like a pandas DataFrame except for the addition of a ``geometry`` columns which holds shapely geometries, along with metadata

In [ ]:
import geopandas as gpd
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

GeoViews ``Path``, ``Contours`` and ``Polygons`` Elements natively support projecting and plotting of
geopandas DataFrames using both ``matplotlib`` and ``bokeh`` plotting extensions. We will load the example dataset of the world which also includes some additional data about each country:

In [ ]:
%%opts Polygons [width=600 height=500 tools=['hover']] (cmap='viridis')
gv.Polygons(world, vdims=['gdp_md_est', 'continent', 'name']).redim.range(Latitude=(-60, 90))

#### Shapes

The ``gv.Shape`` object wraps around any shapely geometry, allowing finer grained control over each polygon. We can, for example select one particular geometry in the geopandas dataframe and display it separately by wrapping it in the ``Shape`` element:

In [ ]:
usa = world[world.name=='United States'].geometry.iloc[0]
gv.Shape(usa)